# Quicktour of PEFT

PEFT offers parameter-efficient methods for finetuning large pretrained models. The traditional paradigm is to finetune all of a model’s parameters for each downstream task, but this is becoming exceedingly costly and impractical because of the enormous number of parameters in models today. Instead, it is more efficient to train a smaller number of prompt parameters or use a reparametrization method like low-rank adaptation (LoRA) to reduce the number of trainable parameters.

This quicktour will show you PEFT’s main features and how you can train or run inference on large models that would typically be inaccessible on consumer devices.

## Import necessary libraries

In [ ]:
from peft import LoraConfig, TaskType, get_peft_model
from transformers import AutoModelForSeq2SeqLM, TrainingArguments, Trainer
from huggingface_hub import notebook_login

## Define the configuration for LoRA

In [ ]:
peft_config = LoraConfig(task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1)

## Load the base model

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("bigscience/mt0-large")

## Wrap the base model with the PEFT configuration

In [ ]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

Out of bigscience/mt0-large’s 1.2B parameters, you’re only training 0.19% of them!

## Set up training arguments

In [ ]:
training_args = TrainingArguments(
    output_dir="alex/mt0-large-lora",
    learning_rate=1e-3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

## Initialize the Trainer and start training

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

## Save the trained model

In [ ]:
model.save_pretrained("output_dir")

## Optionally, push the model to the Hugging Face Hub

In [ ]:
notebook_login()
model.push_to_hub("your-name/bigscience/mt0-large-lora")